# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust-nera,64.5667,143.2000,16.69,45,50,2.72,RU,1689696792
1,1,glace bay,46.1969,-59.9570,27.34,84,75,9.26,CA,1689696792
2,2,invercargill,-46.4000,168.3500,9.03,77,35,4.46,NZ,1689696792
3,3,klintsy,52.7602,32.2393,18.62,73,10,6.56,RU,1689696792
4,4,utrik,11.2278,169.8474,27.88,81,62,5.85,MH,1689696792


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
map_plot_1 =city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size= "Humidity",
    geo = True,
    tiles = "OSM",
    frame_width =700,
    frame_height =500,
    scale = 1,
    color = "City"
)
# YOUR CODE HERE

# Display the map

map_plot_1

# YOUR CODE HERE

C:\Users\asma1\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
temp_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27)]
wind_df = temp_df[temp_df["Wind Speed"] < 4.5]
weather_df = wind_df[wind_df["Cloudiness"] == 0]
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,70,jinchang,38.4953,102.1739,26.95,25,0,1.34,CN,1689696810
140,140,senanga,-16.1167,23.2667,21.95,22,0,1.14,ZM,1689696825
265,265,opuwo,-18.0607,13.8400,23.90,12,0,3.44,NaN,1689696859
410,410,urganch,41.5500,60.6333,25.97,57,0,3.09,UZ,1689696890
458,458,usakos,-22.0000,15.6000,23.34,12,0,1.44,NaN,1689696897
459,459,estevan,49.1334,-102.9842,21.11,64,0,3.60,CA,1689696898
520,520,hohhot,40.8106,111.6522,26.62,56,0,2.00,CN,1689696921


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = weather_df.drop(columns = ["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"]).copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", 0)

# Display sample data
# YOR CODE HERE
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
70,jinchang,38.4953,102.1739,25,CN,0
140,senanga,-16.1167,23.2667,22,ZM,0
265,opuwo,-18.0607,13.8400,12,NaN,0
410,urganch,41.5500,60.6333,57,UZ,0
458,usakos,-22.0000,15.6000,12,NaN,0



### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories="accommodation.hotel"
limit= 1
params = {
    "categories":categories,
    "limit":limit,   
    "apiKey":geoapify_key    
}

    # YOUR CODE HERE

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    Lat = row["Lat"] 
    Lon = row["Lng"] 
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{Lon},{Lat},{radius}"
    params["bias"] = f"proximity:{Lon},{Lat}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jinchang - nearest hotel: No hotel found
senanga - nearest hotel: Senanga Safari Lodge
opuwo - nearest hotel: Ohakane Lodge
urganch - nearest hotel: Khorezm Palace
usakos - nearest hotel: No hotel found
estevan - nearest hotel: Days Inn
hohhot - nearest hotel: Yue Jia Budget Hotel (foreigners approved)


,City,Lat,Lng,Humidity,Country,Hotel Name
70,jinchang,38.4953,102.1739,25,CN,No hotel found
140,senanga,-16.1167,23.2667,22,ZM,Senanga Safari Lodge
265,opuwo,-18.0607,13.8400,12,NaN,Ohakane Lodge
410,urganch,41.5500,60.6333,57,UZ,Khorezm Palace
458,usakos,-22.0000,15.6000,12,NaN,No hotel found
459,estevan,49.1334,-102.9842,64,CA,Days Inn
520,hohhot,40.8106,111.6522,56,CN,Yue Jia Budget Hotel (foreigners approved)


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = "all"
)
# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)